In [25]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist, cifar10, cifar100
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [26]:
# Custom activation functions
def penalized_tanh(x, alpha=0.25):  # alpha is a hyperparameter (default 0.25)
    return tf.where(x >= 0, tf.tanh(x), alpha * tf.tanh(x))

def eliSH(x):
    return x * tf.nn.sigmoid(x) * (1 + tf.nn.tanh(x))

def mish(x):
    return x * tf.nn.tanh(K.softplus(x))  # softplus(x) = log(1 + exp(x))

def rsigelu(x):
    return x * tf.nn.sigmoid(x) + tf.nn.elu(x)

def tanh_exp(x):
    return x * tf.tanh(K.exp(x))

def pflu(x, alpha=0.1, beta=1.0):
    return tf.where(x >= 0, x + beta, alpha * (tf.exp(x) - 1))

def gcu(x):
    return x * tf.cos(x)

def hcLSH(x):
    return x * tf.nn.sigmoid(x) * K.log(K.sigmoid(x) + K.epsilon())


In [27]:
activation_functions = {
    "relu": tf.nn.relu,
    "tanh": tf.nn.tanh,
    "sigmoid": tf.nn.sigmoid,
     "HcLSH": hcLSH
    # "custom_1": custom_activation_1,
    # "custom_2": custom_activation_2
}

In [28]:
def load_svhn():
    train_data = loadmat("path/to/train_32x32.mat")
    test_data = loadmat("path/to/test_32x32.mat")

    X_train = np.moveaxis(train_data["X"], -1, 0)  # Move axis to match TF format
    y_train = train_data["y"].flatten() % 10  # Normalize labels (SVHN uses 1-10)
    
    X_test = np.moveaxis(test_data["X"], -1, 0)
    y_test = test_data["y"].flatten() % 10

    return X_train, X_test, y_train, y_test


In [29]:
def load_dataset(dataset_name):
    if dataset_name == "fashion_mnist":
        (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
        X_train = X_train[..., np.newaxis]  # Add channel dimension
        X_test = X_test[..., np.newaxis]

    elif dataset_name == "cifar10":
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()

    elif dataset_name == "cifar100":
        (X_train, y_train), (X_test, y_test) = cifar100.load_data()

    elif dataset_name == "svhn":
        X_train, X_test, y_train, y_test = load_svhn()

    else:
        raise ValueError("Dataset not supported!")

    # Normalize data
    X_train, X_test = X_train / 255.0, X_test / 255.0

    return X_train, X_test, y_train, y_test

In [30]:
def create_cnn_model(input_shape, activation_name, num_classes):
    activation_func = activation_functions[activation_name]

    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), padding="same", input_shape=input_shape),
        Lambda(activation_func),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=64, kernel_size=(3, 3), padding="same"),
        Lambda(activation_func),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(256),
        Lambda(activation_func),
        Dropout(0.1),
        Dense(num_classes, activation="softmax")  # Multi-class classification
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [31]:
selected_dataset = "fashion_mnist"  # Change to "cifar10", "cifar100", "svhn"
selected_activation = "HcLSH"  # Change to test different activations

X_train, X_test, y_train, y_test = load_dataset(selected_dataset)

num_classes = len(np.unique(y_train))
model = create_cnn_model(X_train.shape[1:], selected_activation, num_classes)
model.summary()

model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

loss, acc = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {acc}")


NameError: Exception encountered when calling layer "lambda_3" (type Lambda).

name 'K' is not defined

Call arguments received by layer "lambda_3" (type Lambda):
  • inputs=tf.Tensor(shape=(None, 28, 28, 32), dtype=float32)
  • mask=None
  • training=None